In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

Este Notebook adiciona colunas ao dataset 'monthly_sales_44',  
referente às vendas mensais da loja escolhida '44' do dataset  
favorita-grocery-sales-forecasting disponível no Kaggle.  
A loja fica em Quito - Pichincha.

Também adiciona as médias dos preços de petróleo.

In [2]:
with open('database/monthly_sales_44.pkl', 'rb') as f:
    df = pickle.load(f)

df.head()

,item_nbr,year,month,onpromotion_anytime_in_month,unit_sales
0,96995,2013,1,True,38.0
1,96995,2013,2,True,23.0
2,96995,2013,3,True,34.0
3,96995,2013,4,True,38.0
4,96995,2013,5,True,59.0


In [3]:
# merges year and month, convert month to its respective name (ex.: 1 == JAN, 2 == FEV)
df['year_month'] = df['year'].astype(str) + '-' + df['month'].astype(str)

df

,item_nbr,year,month,onpromotion_anytime_in_month,unit_sales,year_month
0,96995,2013,1,True,38.0,2013-1
1,96995,2013,2,True,23.0,2013-2
2,96995,2013,3,True,34.0,2013-3
3,96995,2013,4,True,38.0,2013-4
4,96995,2013,5,True,59.0,2013-5
...,...,...,...,...,...,...
138107,2124052,2016,11,True,55.0,2016-11
138108,2124052,2017,7,True,396.0,2017-7
138109,2124052,2017,8,True,278.0,2017-8
138110,2127114,2017,7,True,78.0,2017-7


In [4]:
remove_idx_count = 0
for item_nbr in df['item_nbr'].unique():
    df_item = df[df['item_nbr'] == item_nbr]
    #checks if the item has less than 50 months of data, 
    # counts the total proportion we would have to remove from the data
    if df_item.shape[0] < 56:
        remove_idx_count += df_item.shape[0]
        df = df.drop(df_item.index)
print(f'removed {remove_idx_count} / {df.shape[0] + remove_idx_count} ({remove_idx_count/(df.shape[0]+remove_idx_count)*100:.2f}%) rows')

removed 80712 / 138112 (58.44%) rows


In [5]:
len(df)

57400

adding item category

In [6]:
df_items = pd.read_csv('database/favorita-forecasting/items.csv')
df_items

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1
...,...,...,...,...
4095,2132318,GROCERY I,1002,0
4096,2132945,GROCERY I,1026,0
4097,2132957,GROCERY I,1068,0
4098,2134058,BEVERAGES,1124,0


In [7]:
df = df.merge(df_items, on='item_nbr', how='left')
df.head()

,item_nbr,year,month,onpromotion_anytime_in_month,unit_sales,year_month,family,class,perishable
0,103520,2013,1,True,201.0,2013-1,GROCERY I,1028,0
1,103520,2013,2,True,179.0,2013-2,GROCERY I,1028,0
2,103520,2013,3,True,296.0,2013-3,GROCERY I,1028,0
3,103520,2013,4,True,212.0,2013-4,GROCERY I,1028,0
4,103520,2013,5,True,172.0,2013-5,GROCERY I,1028,0


## Oil Monthly

In [8]:
df_oil = pd.read_csv('database/favorita-forecasting/oil.csv')
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [9]:
# merge all months one by one
df_oil['year'] = df_oil['date'].apply(lambda x: x.split('-')[0])
df_oil['month'] = df_oil['date'].apply(lambda x: x.split('-')[1])

# merges each month into a single row
new_df_oil = df_oil.groupby(['year', 'month']).agg({'dcoilwtico': 'mean'}).reset_index()
new_df_oil['year_month'] = new_df_oil['year'] + '-' + new_df_oil['month']
new_df_oil.drop(['year', 'month'], axis=1, inplace=True)
new_df_oil.head()

,dcoilwtico,year_month
0,94.756667,2013-01
1,95.308947,2013-02
2,92.938500,2013-03
3,92.021364,2013-04
4,94.509545,2013-05


In [10]:
# merges oil prices into the main df by year_month
df['year_month'] = df['year_month'].apply(lambda x: f"{x.split('-')[0]}-{int(x.split('-')[1]):02d}")

df = df.merge(new_df_oil, on='year_month', how='left')

In [11]:
df = df.rename(columns={'dcoilwtico': 'oil_price', 'onpromotion_anytime_in_month' : 'onpromotion'})

In [12]:
# change perishable from 0/1 to False/True
df['perishable'] = df['perishable'].astype(bool)

In [13]:
with open('database/df.pkl', 'wb') as f:
    pickle.dump(df, f)